In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import lightgbm as lgb

from otb.config import ROOT_DIR, DATASETS_FP, TASKS_FP, CACHE_DIR
from otb import CACHE
from otb.tasks import Tasks
from otb.dataset import Dataset

In [2]:
%matplotlib inline

In [3]:
os.getcwd()

'c:\\Users\\cjellen.REDMOND\\Documents\\GitHub\\ot-bench'

### setup (might not be needed?)

In [5]:
CACHE._cache

{}

We might want to move from a system in which tasks are first-class to one in which experiments are first class?

### load the tasks

In [6]:
tasks = Tasks()

List all supported tasks

In [7]:
tasks.list_tasks()

{'regression.mlo_cn2.dropna.Cn2_15m', 'regression.mlo_cn2.full.Cn2_15m'}

Get details for the `mlo_cn2` regression task, with missing values dropped

In [8]:
task = tasks.get_task("regression.mlo_cn2.dropna.Cn2_15m")

Get the training data

In [9]:
c = CACHE

In [10]:
c.available_datasets()

['mlo_cn2']

In [11]:
df = Dataset("mlo_cn2")

In [12]:
task

{'description': 'Regression task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
 'description_long': 'This dataset evaluates regression approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected at the Mauna Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are used to evaluate prediction accuracy under the root-mean square error metric.',
 'ds_name': 'mlo_cn2',
 'train_idx': ['0:10367'],
 'test_idx': ['10367:13943'],
 'dropna': True,
 'log_transform': True,
 'eval_metrics': ['root_mean_squared_error',
  'r2_score',
  'mean_absolute_error',
  'mean_absolute_percentage_error'],
 'target': 'Cn2_15m',
 'remove': ['base_time', 'Cn2_6m', 'Cn2_15m', 'Cn2_25m']}

In [13]:
X_train, y_train = df.get_train(task=task)

Train your model

In [14]:
model = lgb.LGBMRegressor()

In [15]:
model.fit(X_train, y_train)

LGBMRegressor()

In [16]:
X_test, y_test = df.get_test(task=task)

In [20]:
y_test_pred = model.predict(X_test)

In [21]:
from sklearn.metrics import r2_score

r2_score(y_test_pred, y_test)

0.9020957932915938

Compute metrics on the evaluation set

Compare against benchmarks